## Building an Image Generation App with Gradio

In this exercise, we will use OpenAI's [DALL-E](https://openai.com/dall-e-2) [API](https://platform.openai.com/docs/guides/images) to build a text-to-image generator.

#### Setup
1. Create an [OpenAI account](https://platform.openai.com/signup). This will give you some free credits to play with.
2. Generate a new secret key [here](https://platform.openai.com/account/api-keys) and copy the value into the cell below. Make sure to note it down somewhere secure, because this will not be displayed after the first time.

In [ ]:
import gradio as gr
import openai

openai.api_key = '' # paste your API key here

#### API call
We will use the OpenAI [image generations](https://platform.openai.com/docs/api-reference/images/create) endpoint to generate an image. Generated images can have a size of 256x256, 512x512, or 1024x1024 pixels. Smaller sizes are faster to generate. You can request 1-10 images at a time using the `n` parameter.

In [ ]:
def generate_image(prompt: str):
    response = openai.Image.create(user='testing', prompt=prompt, n=1)
    return response["data"][0]["url"]

In [ ]:
demo = gr.Interface(fn=generate_image, inputs="text", outputs="image")

demo.launch()

#### Prompt Engineering
Now let's try building more complex prompts using fields with additional data. To help the user do this, we will use dropdowns with various options for image type, aesthetics, framing and emotion.

#### Using Blocks
[`Block`](https://www.gradio.app/docs/blocks) is Gradio's low-level class that is more flexible and supports more customisable web applications and demos than `Interface`. With Blocks, you have more control over:
1. the layout of components
2. the events that trigger the execution of functions 
3. data flows (e.g. inputs can trigger outputs, which can trigger the next level of outputs). 
Blocks also offers ways to group together related demos such as with tabs.

To use Blocks, we shall: 
1. Create a Blocks object and use it as a context (using the `with` statement)
2. Define the layouts, components, and events within the context. Note that components created here are automatically added to the app, and are rendered vertically in the order that they are defined.
3. Finally, call `launch()` to launch the demo.

In [ ]:
def create_combined_prompt(prompt: str, image_type: str, image_aesthetics: str, image_framing: str, image_emotion: str):
    return f"{image_type} of {prompt}, {image_aesthetics}, {image_framing}, {image_emotion}"

def generate_prompt_and_image(prompt: str, image_type: str, image_aesthetics: str, image_framing: str, image_emotion: str):
    combined_prompt = create_combined_prompt(prompt, image_type, image_aesthetics, image_framing, image_emotion)
    return generate_image(combined_prompt)

In [ ]:

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt_box = gr.Textbox(label="prompt")
            image_type_choices = gr.Dropdown(label="Type of image", choices=["photograph", "oil painting", "watercolour painting", "anime", "cartoon image", "pop art", "photorealistic", "3d art", "vector art", "collage", "scientific diagram", "tattoo", "pixel art", "comic book art"])
            image_aesthetic_choices = gr.Dropdown(label="Aesthetics of the image", choices=["art deco", "post-apocalyptic", "sci-fi", "steampunk", "cyberpunk", "autumn", "avant-garde", "emo", "futuristic", "goth", "kawaii", "minimalism", "medieval", "new age", "surrealism", "retro", "vintage", "western"])
            image_framing_choices = gr.Dropdown(label="How the image is framed", choices=["close-up", "long shot", "wide angle", "overhead view", "aerial view"])
            image_emotional_choices = gr.Dropdown(label="Emotions evoked by image", choices=["peaceful", "tranquil", "vibrant", "dynamic", "expressive", "romantic", "sombre", "gloomy", "pale", "ominous", "dark", "sinister", "pastel"])
            generate_btn = gr.Button("Generate")
        with gr.Column():
            output_img = gr.Image()
            generate_btn.click(fn=generate_prompt_and_image, inputs=[
                prompt_box, 
                image_type_choices, 
                image_aesthetic_choices, 
                image_framing_choices, 
                image_emotional_choices
            ], outputs=output_img)

demo.launch()      

#### Multiple Control Flows

Can you extend this demo to do the following?

Let's say that as a user, I want to view the final prompt before submitting it to the API. I may choose to edit it multiple times until I'm happy with the final result.

Add a button, which, when clicked, displays the combined prompt - but does not generate an image. Only when the Generate button is hit, is the request to generate an image submitted.